In [6]:
import csv
import json
from datetime import datetime
"""
    3. *job_compatibility*
        From "articoli_macchine.json" we retrieve the associations
    4. *machine_compatibility*
        From "macchine_articoli.json" we retrieve the associations
    """
job_compatibility = {} #
machine_compatibility = {} #DONE
id_articles = {} #Dictionary of associations for giving at each articles a numerical id

with open('data/utils/articoli_macchine.json') as file:
    job_compatibility = json.load(file)

with open('data/utils/macchine_articoli.json') as file:
    machine_compatibility = json.load(file)

for idx, article in enumerate(job_compatibility):
    id_articles[article] = idx
print(f'id_articles: {id_articles}')
print(f'number of machines: {len(machine_compatibility)}\nnumber of articles: {len(job_compatibility)}')


def date_hours_parser(start_date, due_date):
    """
    Parser for start and due date from strings to hours
    """
    # Parse input dates
    start = datetime.strptime(start_date, "%Y-%m-%d")
    due = datetime.strptime(due_date, "%Y-%m-%d")
    
    # Get current date and time
    current_datetime = datetime.now()

    # Calculate differences in hours
    start_hours = int((start - current_datetime).total_seconds() // 3600)
    due_hours = int((due - current_datetime).total_seconds() // 3600)

    if start_hours < 0:
        start_hours = 0

    return start_hours, due_hours


id_articles: {'2277DN mini': 0, '2011DN mini': 1, '2271DN 20700 mini': 2, '749DN mini': 3, 'X22-1703DN minispole': 4, 'X23-3399S+ZF': 5, '2270s+z rpm 19800': 6, '2270S+ZN 19800': 7, 'X23-2299V5ZF': 8, '7032S+ZN': 9, '2270s+z rpm 21000': 10, '2250S+ZN': 11, '31002DN1': 12, 'X24-4401S+ZF2': 13, 'X23-4493S+ZF': 14, 'X21-2291SF': 15, 'X21-2291sf': 16, '2270S+ZN mini': 17, 'X21-2099S+ZN mini': 18, 'X24-2001S+ZN': 19, '13007ZN/F-14500': 20, '28503ZN': 21, '2246ZN 10800': 22, 'X23-2280S+ZN': 23, 'X20-2222DN2 mini': 24, '33002DN': 25, 's22-700011DN': 26, 'S600A04DN': 27, 'S600A03DN': 28, '33001DN2': 29, 'X23-8397DN': 30, '31003DN220274': 31, 'X21-39599V7DN0332': 32, '62002DN': 33, '39502DN-14446': 34, '39502dn': 35, '62002DN2': 36, 'X21-20099DN2': 37, 'S180001DN': 38, 'X23-47097DN366': 39, 'X24-31001DN366': 40, 'X23-47097DN1': 41, '13005ZF-14000': 42, '2265SN2': 43, 'X24-2202s+z': 44, '7034ZN': 45, '2275S+ZN-B10A': 46, 'X22-2211V2zf': 47, '4407DN': 48, 'X22-7810S-ZN354 4 ci': 49, 'X22-2203S+ZN

In [5]:
m_info_path = 'data/utils/macchine_info.json'
standard_setup_cost = 1

base_setup_cost = {}

m_info = json.load(open(m_info_path))
fuses_machines_associations = {machine:m_info[machine]['n_fusi'] for machine in m_info}

for a in job_compatibility:
    for m in fuses_machines_associations:
        base_setup_cost[a,m] = int(float(standard_setup_cost * fuses_machines_associations[m]))

display(m_info)
display(fuses_machines_associations)
display(base_setup_cost)

{'1': {'classe': 'Gruppo1b',
  'n_fusi': '392',
  'mode': 'Doppia',
  'descrizione': '1 motore con rinvio',
  'articoli': ['2011DN',
   '2277DN',
   '2277DN mini',
   '2271DN 20700',
   '2271DN 22000']},
 '2': {'classe': 'Gruppo1a',
  'n_fusi': '448',
  'mode': 'Doppia',
  'descrizione': '1 motore con rinvio',
  'articoli': ['2271DN 22000',
   '2271DN 20700',
   '848DN',
   '2011DN mini',
   '4407DN']},
 '3': {'classe': 'Gruppo1a',
  'n_fusi': '448',
  'mode': 'Doppia',
  'descrizione': '1 motore con rinvio',
  'articoli': ['X22-1703DN',
   '2271DN 20700',
   '2271DN 20700 mini',
   '2271DN 22000',
   '2011DN mini']},
 '4': {'classe': 'Gruppo1a',
  'n_fusi': '448',
  'mode': 'Doppia',
  'descrizione': '1 motore con rinvio',
  'articoli': ['2271DN 20700', '2011DN', '2271DN 22000', '2011DN mini']},
 '5': {'classe': 'Gruppo1a',
  'n_fusi': '448',
  'mode': 'Doppia',
  'descrizione': '1 motore con rinvio',
  'articoli': ['2271DN 20700 mini',
   '2271DN 20700',
   '2011DN',
   '2271DN 22000

{'1': '392',
 '2': '448',
 '3': '448',
 '4': '448',
 '5': '448',
 '6': '256',
 '7': '448',
 '8': '448',
 '9': '448',
 '10': '448',
 '11': '448',
 '12': '448',
 '13': '256',
 '14': '256',
 '15': '256',
 '16': '256',
 '17': '256',
 '18': '256',
 '19': '256',
 '20': '256',
 '21': '256',
 '22': '256',
 '23': '256',
 '24': '256',
 '25': '256',
 '26': '256',
 '27': '256',
 '28': '256',
 '29': '256',
 '30': '256',
 '31': '256',
 '32': '256',
 '33': '256',
 '34': '256',
 '35': '256',
 '36': '256',
 '37': '256',
 '38': '256',
 '39': '256',
 '40': '256',
 '41': '256',
 '42': '256',
 '43': '256',
 '44': '256',
 '45': '256',
 '46': '256',
 '47': '448',
 '48': '448',
 '49': '160',
 '50': '160',
 '51': '264',
 '52': '264',
 '53': '264',
 '54': '264',
 '55': '264',
 '56': '264',
 '57': '264',
 '58': '264',
 '59': '264',
 '60': '264',
 '61': '448',
 '62': '448',
 '63': '448',
 '64': '448',
 '65': '448',
 '66': '448',
 '67': '264',
 '68': '264',
 '69': '264',
 '70': '264',
 '71': '264',
 '72': '264'}

{('2277DN mini', '1'): 392,
 ('2277DN mini', '2'): 448,
 ('2277DN mini', '3'): 448,
 ('2277DN mini', '4'): 448,
 ('2277DN mini', '5'): 448,
 ('2277DN mini', '6'): 256,
 ('2277DN mini', '7'): 448,
 ('2277DN mini', '8'): 448,
 ('2277DN mini', '9'): 448,
 ('2277DN mini', '10'): 448,
 ('2277DN mini', '11'): 448,
 ('2277DN mini', '12'): 448,
 ('2277DN mini', '13'): 256,
 ('2277DN mini', '14'): 256,
 ('2277DN mini', '15'): 256,
 ('2277DN mini', '16'): 256,
 ('2277DN mini', '17'): 256,
 ('2277DN mini', '18'): 256,
 ('2277DN mini', '19'): 256,
 ('2277DN mini', '20'): 256,
 ('2277DN mini', '21'): 256,
 ('2277DN mini', '22'): 256,
 ('2277DN mini', '23'): 256,
 ('2277DN mini', '24'): 256,
 ('2277DN mini', '25'): 256,
 ('2277DN mini', '26'): 256,
 ('2277DN mini', '27'): 256,
 ('2277DN mini', '28'): 256,
 ('2277DN mini', '29'): 256,
 ('2277DN mini', '30'): 256,
 ('2277DN mini', '31'): 256,
 ('2277DN mini', '32'): 256,
 ('2277DN mini', '33'): 256,
 ('2277DN mini', '34'): 256,
 ('2277DN mini', '35'):

In [7]:
class Product (object) :
    '''
    A Product is a request for a certain amount of an certain article
    '''
    def __init__(self, id, article, kg_request, start_date, due_date):
        # Input level Data
        self.id = id
        self.article = article
        self.kg_request = kg_request
        self.start_date = start_date
        self.due_date = due_date

        # Output data (will be filled by the scheduler output)
        self.machine = {}
        self.velocity = {}
        self.num_levate = {}
        self.setup_beg = {}
        self.setup_end = {}
        self.cycle_end = {}
        self.load_beg = {}
        self.load_end = {}
        self.unload_beg = {}
        self.unload_end = {}
    
    def __str__(self) :
        return f"Product {self.id}\n    Article : {self.article}\n    Request : {self.kg_request} Kg\n    Start date : {self.start_date}\n    Due date : {self.due_date}\n---"

class RunningProduct (Product) :
    '''
    A RunningProduct is a some product which is already being processed by some machine
    when the scheduling operation begins
    '''
    def __init__(self, id, article, kg_request, start_date, due_date, machine, operator, velocity, remaining_levate, current_op_type, remaining_time) :
        super().__init__(id, article, kg_request, start_date, due_date)
        # Data related to settings associated to the running product
        self.operator = operator
        self.machine[0] = machine
        self.velocity[0] = velocity
        self.remaining_levate = remaining_levate # amount of levate to finish the cycle

        # states in which phase the product is when the scheduling starts
        #   can have 4 possible values {0, 1, 2, 3} 
        #   associated to a all cycle's possible phases {setup, load, running, unload}
        self.current_op_type = current_op_type
        # remaining time to finish the active production phase
        self.remaining_time = remaining_time

    def __str__(self) :
        return f"RunningProduct {self.id}\n    Article : {self.article}\n    Request : {self.kg_request} Kg\n    Start date : {self.start_date}\n    Due date : {self.due_date}\n    Machine : {self.machine}\n    Operator : {self.operator}\n    Velocity : {self.velocity}\n    Remaining levate : {self.remaining_levate}\n    Current operation type : {self.current_op_type}\n    Remaining time : {self.remaining_time}\n---"


class Schedule(object):
    '''
    A schedule is the final result of the Job Scheduling Problem (JSP).
    '''
    def __init__(self, products):
        # Reference to products class instances, ensuring proper types
        self.products = products
        for _, prod in self.products:
            if not isinstance(prod, (Product, RunningProduct)):
                raise ValueError("Invalid product type")

    def __str__(self):
        output = "Production Schedule:\n"
        for p, prod in self.products :
            output += f"Product : {chr(p+65)}\n"
            for c in prod.setup_beg.keys():
                output += f"    Cycle {c} :\n"
                output += f"        Machine   : {prod.machine[c]}:\n"
                output += f"        Velocity  : {prod.velocity[c]}\n"
                output += f"        Cycle End : {prod.cycle_end[c]}\n"
                output += f"        Setup     : ({prod.setup_beg[c]}, {prod.setup_end[c]})\n"
                for l in range(prod.num_levate[c]) :
                    if (c,l) in prod.load_beg.keys() : 
                        output += f"            Levata [{l}] : ({prod.load_beg[c,l]}, {prod.load_end[c,l]}) => ({prod.unload_beg[c,l]}, {prod.unload_end[c,l]})\n"
            output += "\n"
        
        return output
    
    def __len__(self):
        return len(self.products)

In [30]:

def init_csv_data(common_p_path: str, j_compatibility_path: str, m_compatibility_path: str, m_info_path: str, article_list_path: str, costs = (2,1,1), running_products =  {}):
    """
    - `costs:` a tuple representing the costs of the operation made by the human operators, cost expressed *per fuse*:
        - `base_setup_cost:`
        - `base_load_cost:`
        - `base_unload_cost:`


    1. *common_products*
        The csv gave us:
        - Client name (useless)
        - Product code 
        - Product quantity
        - Insertion date
        - Due date
        for every row, which is a "Product" object and the list of those form the "common_products"
    2. *running_products*
        Assuming an empty list right now
    3. *job_compatibility*
        From "articoli_macchine.json" we retrieve the associations
    4. *machine_compatibility*
        From "macchine_articoli.json" we retrieve the associations
    5. *base_setup_cost*
        Costant taken from input, machine dependent (number of fuses)
    6. *base_load_cost*
        For every product is machine dependent(depends from the number of fusi) 
    7. *base_unload_cost*
        Same as point 6 
    8.  *base_levata_cost*
        For every product, from "lista_articoli" see "ore_levata" 
    9. *standard_levate*
        For every product, form "lista_articoli" see "no_cicli" but is leavta 
    10. *kg_per_levata*
        See from "lista_articoli" the "kg_ora" and the "ore_levata" 
    
"""
    const_setup_cost, const_load_cost, const_unload_cost = costs
    
    # 3. job_compatibility, starts from this for retrieve id_articles
    job_compatibility = json.load(open(j_compatibility_path))

    # 0. id_articles, indexing with a number every article
    # id_articles = {}
    
    # for idx, article in enumerate(job_compatibility):
    #     id_articles[article] = idx
    # print(f'id_articles: {id_articles}')
    

    # 1. common_products
    with open(common_p_path, newline='') as csvfile:
        
        csv_data = csv.reader(csvfile, delimiter=',', quotechar='|')
        common_products = []
        str_out = ""
        for idx, row in enumerate(csv_data):
            if idx > 0:
                str_out = str_out + ', '.join(row) 
                str_out += '\n'
                # print(f'start_date:  {row[3]} - type: {type(row[3])}')
                # print(f'id:  {row[0]} - article: {row[1]}')
                start_date, due_date = date_hours_parser(row[3], row[4])
                # common_products.append(Product(idx, id_articles[row[1]], int(float(row[2])), start_date, due_date))
                common_products.append(Product(idx, row[1], int(float(row[2])), start_date, due_date))
    
    # 2. running_products
    # there is nothing here for now

    # 3. job_compatibility
    job_compatibility = json.load(open(j_compatibility_path))

    # 4. machine_compatibility
    machine_compatibility = json.load(open(m_compatibility_path))

    # 5-6-7. base_setup_cost, base_load_cost, base_unload_cost
    base_setup_cost = {}
    base_load_cost = {}
    base_unload_cost = {}

    m_info = json.load(open(m_info_path))
    fuses_machines_associations = {machine:m_info[machine]['n_fusi'] for machine in m_info}

    for a in job_compatibility:
        for m in fuses_machines_associations:
            base_setup_cost[a,m] = int(float(const_setup_cost) * float(fuses_machines_associations[m]))
            base_load_cost[a,m] = int(float(const_load_cost) * float(fuses_machines_associations[m]))
            base_unload_cost[a,m] = int(float(const_unload_cost) * float(fuses_machines_associations[m]))

    # 8-9-10. base_levata_cost, standard_levate, kg_per_levata
    base_levata_cost = {}
    standard_levate = {}
    kg_per_levata = {}

    with open(article_list_path, newline='') as csvfile:
        csv_data = csv.reader(csvfile, delimiter=',', quotechar='"')
        for idx, row in enumerate(csv_data):
            # row[0] is the article code, row[10] is the hours needed for a single "levata"
            if idx > 0:
                try:
                    base_levata_cost[row[0]] = int(float(row[10]))
                    standard_levate[row[0]] = int(float(row[9]))
                    kg_per_levata[row[0]] = int(float(row[9]) * float(row[6]))
                except:
                    print(f'row[0]:{row[0]}, row[6]:{row[6]}, row[9]:{row[9]}, row[10]:{row[10]}')

    print(str_out)
    for i in common_products:
        print(i.__str__())

        
    return common_products, running_products, job_compatibility, machine_compatibility, base_setup_cost, base_load_cost, base_unload_cost, base_levata_cost, standard_levate, kg_per_levata


COMMON_P_PATH = 'data/new_orders.csv'
J_COMPATIBILITY_PATH = 'data/utils/articoli_macchine.json'
M_COMPATIBILITY_PATH = 'data/utils/macchine_articoli.json'
M_INFO_PATH = 'data/utils/macchine_info.json'
ARTICLE_LIST_PATH = 'data/valid/lista_articoli.csv'



In [31]:
# In the article list:
# There are in column "ciclo" some data are structured like (657,5), and the "," causes problems
# 



init_csv_data(COMMON_P_PATH, J_COMPATIBILITY_PATH, M_COMPATIBILITY_PATH, M_INFO_PATH, ARTICLE_LIST_PATH, costs=(1/512, 1/256, 1/256))


FOCENZA FACTORY, 4424S+ZN/F, 220.0, 2024-11-13, 2024-12-09
RADICI, 2275S+ZN-B10A, 4366.666666666667, 2024-11-13, 2025-03-16
FOCENZA FACTORY, 4424S+ZN/F, 220.0, 2024-11-13, 2024-12-09
FROHN GMBH, S600A03DN, 75.0, 2024-11-13, 2024-12-09
CALZIFICIO ZETA, 33001DN2, 100.0, 2024-11-13, 2024-12-09
INVENTEX, S15-160018DN, 587.0, 2024-11-13, 2024-12-03
FALKE, X22-2005V2ZN1, 27.0, 2024-11-13, 2024-12-09
CSP INTERNATIONAL, 1724S+ZN 2cicli, 150.0, 2024-11-13, 2025-01-02
FISHER & PAYKEL, S19-60064V3DN-U427, 80.0, 2024-11-13, 2024-12-09

Product 1
    Article : 4424S+ZN/F
    Request : 220 Kg
    Start date : 0
    Due date : 606
---
Product 2
    Article : 2275S+ZN-B10A
    Request : 4366 Kg
    Start date : 0
    Due date : 2934
---
Product 3
    Article : 4424S+ZN/F
    Request : 220 Kg
    Start date : 0
    Due date : 606
---
Product 4
    Article : S600A03DN
    Request : 75 Kg
    Start date : 0
    Due date : 606
---
Product 5
    Article : 33001DN2
    Request : 100 Kg
    Start date : 0
  

([<__main__.Product at 0x7fa9c0327e20>,
 {},
 {'2277DN mini': [1,
   2,
   3,
   4,
   5,
   7,
   8,
   9,
   10,
   11,
   12,
   47,
   48,
   61,
   62,
   63,
   64,
   65,
   66],
  '2011DN mini': [2,
   3,
   4,
   5,
   7,
   8,
   9,
   10,
   11,
   12,
   47,
   48,
   61,
   62,
   63,
   64,
   65,
   66],
  '2271DN 20700 mini': [2,
   3,
   4,
   5,
   7,
   8,
   9,
   10,
   11,
   12,
   47,
   48,
   61,
   62,
   63,
   64,
   65,
   66],
  '749DN mini': [6, 49, 50],
  'X22-1703DN minispole': [2,
   3,
   4,
   5,
   7,
   8,
   9,
   10,
   11,
   12,
   47,
   48,
   61,
   62,
   63,
   64,
   65,
   66],
  'X23-3399S+ZF': [13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   33,
   34,
   35,
   36,
   44,
   45,
   46],
  '2270s+z rpm 19800': [13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   33,
   34,
   35,
   36],
  '2270S+ZN 19800': [13,


In [42]:
from datetime import datetime

def calculate_hours_difference(start_date, due_date):
    # Parse input dates
    start = datetime.strptime(start_date, "%Y-%m-%d")
    due = datetime.strptime(due_date, "%Y-%m-%d")
    
    # Get current date and time
    current_datetime = datetime.now()

    # Calculate differences in hours
    start_hours = int((start - current_datetime).total_seconds() // 3600)
    due_hours = int((due - current_datetime).total_seconds() // 3600)

    if start_hours < 0:
        start_hours = 0

    return start_hours, due_hours

# Example usage
start_date = "2024-11-12"
due_date = "2024-12-01"
hours_until_start, hours_until_due = calculate_hours_difference(start_date, due_date)

print(f"Hours until start date: {hours_until_start}")
print(f"Hours until due date: {hours_until_due}")


Hours until start date: 0
Hours until due date: 436
